### Chapter 18: Communication and Synchronization

Communication and synchronization are fundamental aspects of SystemVerilog testbenches, enabling coordination between different processes, threads, and verification components. This chapter covers the essential mechanisms for inter-process communication and synchronization.

#### Mailboxes for Inter-Process Communication

Mailboxes provide a mechanism for passing data between processes in a FIFO (First-In-First-Out) manner. They are particularly useful in testbench environments where different components need to exchange information asynchronously.

##### Basic Mailbox Operations

```systemverilog
// Mailbox declaration
mailbox mb;

// Parameterized mailbox for type safety
mailbox #(int) int_mb;
mailbox #(packet_t) packet_mb;

// Basic mailbox operations
initial begin
    mb = new();  // Create unbounded mailbox
    
    // Put data into mailbox
    mb.put(42);
    mb.put("Hello");
    
    // Get data from mailbox
    int value;
    string msg;
    mb.get(value);  // Blocking get
    mb.get(msg);
    
    $display("Received: %0d, %s", value, msg);
end
```

##### Bounded Mailboxes

```systemverilog
// Bounded mailbox with size limit
mailbox #(int) bounded_mb;

initial begin
    bounded_mb = new(5);  // Mailbox with capacity of 5
    
    // Fill the mailbox
    for (int i = 0; i < 5; i++) begin
        bounded_mb.put(i);
        $display("Put %0d into mailbox", i);
    end
    
    // This will block until space is available
    fork
        bounded_mb.put(99);  // Will block
    join_none
    
    // Make space by getting an item
    int data;
    bounded_mb.get(data);
    $display("Got %0d from mailbox", data);
end
```

##### Non-blocking Mailbox Operations

```systemverilog
mailbox #(int) nb_mb;

initial begin
    nb_mb = new(3);
    
    // Non-blocking put
    if (nb_mb.try_put(10))
        $display("Successfully put 10");
    else
        $display("Mailbox full, couldn't put 10");
    
    // Non-blocking get
    int data;
    if (nb_mb.try_get(data))
        $display("Got %0d from mailbox", data);
    else
        $display("Mailbox empty, nothing to get");
    
    // Peek at next item without removing it
    if (nb_mb.try_peek(data))
        $display("Next item is %0d", data);
end
```

##### Producer-Consumer Example

```systemverilog
typedef struct {
    int id;
    string data;
    bit [31:0] timestamp;
} transaction_t;

mailbox #(transaction_t) txn_mb;

// Producer process
task producer();
    transaction_t txn;
    for (int i = 0; i < 10; i++) begin
        txn.id = i;
        txn.data = $sformatf("Transaction_%0d", i);
        txn.timestamp = $time;
        txn_mb.put(txn);
        $display("[%0t] Producer: Generated transaction %0d", $time, i);
        #10;
    end
endtask

// Consumer process
task consumer();
    transaction_t txn;
    forever begin
        txn_mb.get(txn);
        $display("[%0t] Consumer: Processing transaction %0d: %s", 
                 $time, txn.id, txn.data);
        #5;  // Processing time
    end
endtask

initial begin
    txn_mb = new();
    fork
        producer();
        consumer();
    join_any
    #200;
    $finish;
end
```

#### Semaphores for Resource Sharing

Semaphores control access to shared resources by maintaining a count of available resources. They are essential for preventing race conditions and managing resource allocation.

##### Basic Semaphore Operations

```systemverilog
// Semaphore declaration
semaphore sem;

initial begin
    sem = new(2);  // Semaphore with 2 keys (resources)
    
    fork
        // Process 1
        begin
            sem.get(1);  // Acquire 1 key
            $display("[%0t] Process 1: Got semaphore", $time);
            #20;
            sem.put(1);  // Release 1 key
            $display("[%0t] Process 1: Released semaphore", $time);
        end
        
        // Process 2
        begin
            #5;
            sem.get(1);  // Acquire 1 key
            $display("[%0t] Process 2: Got semaphore", $time);
            #15;
            sem.put(1);  // Release 1 key
            $display("[%0t] Process 2: Released semaphore", $time);
        end
        
        // Process 3 (will wait)
        begin
            #10;
            sem.get(1);  // Will block until resource available
            $display("[%0t] Process 3: Got semaphore", $time);
            #10;
            sem.put(1);
            $display("[%0t] Process 3: Released semaphore", $time);
        end
    join
end
```

##### Non-blocking Semaphore Operations

```systemverilog
semaphore resource_sem;

task try_access_resource(int process_id);
    if (resource_sem.try_get(1)) begin
        $display("[%0t] Process %0d: Acquired resource", $time, process_id);
        #($urandom_range(10, 30));  // Use resource
        resource_sem.put(1);
        $display("[%0t] Process %0d: Released resource", $time, process_id);
    end else begin
        $display("[%0t] Process %0d: Resource busy, trying later", $time, process_id);
        #5;
        try_access_resource(process_id);  // Retry
    end
endtask

initial begin
    resource_sem = new(1);  // Single resource
    
    fork
        try_access_resource(1);
        try_access_resource(2);
        try_access_resource(3);
    join
end
```

##### Multiple Resource Acquisition

```systemverilog
semaphore multi_sem;

task acquire_multiple_resources(int count, int process_id);
    $display("[%0t] Process %0d: Requesting %0d resources", $time, process_id, count);
    multi_sem.get(count);  // Acquire multiple resources
    $display("[%0t] Process %0d: Acquired %0d resources", $time, process_id, count);
    #20;  // Use resources
    multi_sem.put(count);  // Release all resources
    $display("[%0t] Process %0d: Released %0d resources", $time, process_id, count);
endtask

initial begin
    multi_sem = new(5);  // 5 available resources
    
    fork
        acquire_multiple_resources(2, 1);
        acquire_multiple_resources(3, 2);
        acquire_multiple_resources(2, 3);
    join
end
```

#### Events for Synchronization

Events provide a mechanism for synchronization between processes. They can be triggered and waited upon, enabling coordination of activities across different threads.

##### Basic Event Operations

```systemverilog
// Event declaration
event start_event;
event done_event;

// Triggering events
initial begin
    #10;
    -> start_event;  // Trigger event
    $display("[%0t] Start event triggered", $time);
    
    wait(done_event.triggered);
    $display("[%0t] Done event detected", $time);
end

// Waiting for events
initial begin
    @(start_event);  // Wait for event
    $display("[%0t] Received start signal", $time);
    #15;
    -> done_event;   // Trigger completion
end
```

##### Event Synchronization Patterns

```systemverilog
event phase1_done, phase2_done, all_done;

// Multiple processes synchronized by events
initial begin
    fork
        // Process A
        begin
            $display("[%0t] Process A: Starting phase 1", $time);
            #20;
            $display("[%0t] Process A: Phase 1 complete", $time);
            -> phase1_done;
            
            @(phase2_done);  // Wait for phase 2
            $display("[%0t] Process A: Continuing after phase 2", $time);
            #10;
            -> all_done;
        end
        
        // Process B
        begin
            @(phase1_done);  // Wait for phase 1
            $display("[%0t] Process B: Starting phase 2", $time);
            #15;
            $display("[%0t] Process B: Phase 2 complete", $time);
            -> phase2_done;
            
            @(all_done);
            $display("[%0t] Process B: All phases complete", $time);
        end
    join
end
```

##### Event Arrays and Dynamic Events

```systemverilog
// Array of events
event worker_done[4];
event all_workers_done;

initial begin
    // Start multiple workers
    fork
        for (int i = 0; i < 4; i++) begin
            automatic int worker_id = i;
            fork
                begin
                    $display("[%0t] Worker %0d: Starting", $time, worker_id);
                    #($urandom_range(10, 30));
                    $display("[%0t] Worker %0d: Done", $time, worker_id);
                    -> worker_done[worker_id];
                end
            join_none
        end
    join_none
    
    // Wait for all workers
    fork
        begin
            for (int i = 0; i < 4; i++) begin
                @(worker_done[i]);
                $display("[%0t] Worker %0d completion detected", $time, i);
            end
            -> all_workers_done;
        end
    join_none
    
    @(all_workers_done);
    $display("[%0t] All workers completed", $time);
end
```

#### Fork-Join Constructs

Fork-join constructs enable parallel execution of processes with different synchronization behaviors.

##### Fork-Join Types

```systemverilog
initial begin
    $display("=== Fork-Join Demo ===");
    
    // fork-join: Wait for ALL processes to complete
    $display("[%0t] Starting fork-join", $time);
    fork
        begin #10; $display("[%0t] Process 1 done", $time); end
        begin #20; $display("[%0t] Process 2 done", $time); end
        begin #15; $display("[%0t] Process 3 done", $time); end
    join
    $display("[%0t] All processes in fork-join completed", $time);
    
    // fork-join_any: Wait for ANY process to complete
    $display("[%0t] Starting fork-join_any", $time);
    fork
        begin #10; $display("[%0t] Fast process done", $time); end
        begin #30; $display("[%0t] Slow process done", $time); end
    join_any
    $display("[%0t] First process in fork-join_any completed", $time);
    
    // fork-join_none: Don't wait for any process
    $display("[%0t] Starting fork-join_none", $time);
    fork
        begin #5; $display("[%0t] Background process 1", $time); end
        begin #8; $display("[%0t] Background process 2", $time); end
    join_none
    $display("[%0t] Continuing without waiting", $time);
    
    #40;  // Wait to see background processes
    $display("[%0t] End of demo", $time);
end
```

##### Nested Fork-Join

```systemverilog
initial begin
    $display("=== Nested Fork-Join ===");
    
    fork
        // Parallel branch 1
        begin
            $display("[%0t] Branch 1: Starting", $time);
            fork
                begin #5; $display("[%0t] Branch 1.1 done", $time); end
                begin #8; $display("[%0t] Branch 1.2 done", $time); end
            join
            $display("[%0t] Branch 1: All sub-processes done", $time);
        end
        
        // Parallel branch 2
        begin
            $display("[%0t] Branch 2: Starting", $time);
            fork
                begin #12; $display("[%0t] Branch 2.1 done", $time); end
                begin #6;  $display("[%0t] Branch 2.2 done", $time); end
            join_any
            $display("[%0t] Branch 2: First sub-process done", $time);
        end
    join
    
    $display("[%0t] All main branches completed", $time);
end
```

##### Process Control with Fork-Join

```systemverilog
process main_proc;
process monitor_proc;

initial begin
    fork
        // Main process
        begin
            main_proc = process::self();
            $display("[%0t] Main process starting", $time);
            for (int i = 0; i < 100; i++) begin
                #1;
                if (i == 50) begin
                    $display("[%0t] Main process halfway", $time);
                end
            end
            $display("[%0t] Main process completed", $time);
        end
        
        // Monitor process
        begin
            monitor_proc = process::self();
            #25;
            $display("[%0t] Monitor: Killing main process", $time);
            main_proc.kill();
        end
    join_any
    
    $display("[%0t] Fork-join_any completed", $time);
end
```

#### Process Control

SystemVerilog provides mechanisms to control process execution, including suspension, resumption, and termination.

##### Process Handle Operations

```systemverilog
process worker_process;
process monitor_process;

task worker_task();
    worker_process = process::self();
    for (int i = 0; i < 20; i++) begin
        $display("[%0t] Worker: Step %0d", $time, i);
        #5;
        
        // Check if process should be suspended
        if (worker_process.status() == process::SUSPENDED) begin
            $display("[%0t] Worker: Process suspended", $time);
            wait(worker_process.status() != process::SUSPENDED);
            $display("[%0t] Worker: Process resumed", $time);
        end
    end
    $display("[%0t] Worker: Task completed", $time);
endtask

initial begin
    fork
        worker_task();
        
        // Control process
        begin
            monitor_process = process::self();
            #30;
            $display("[%0t] Monitor: Suspending worker", $time);
            worker_process.suspend();
            
            #20;
            $display("[%0t] Monitor: Resuming worker", $time);
            worker_process.resume();
            
            #40;
            if (worker_process.status() == process::RUNNING) begin
                $display("[%0t] Monitor: Killing worker", $time);
                worker_process.kill();
            end
        end
    join_any
end
```

##### Process Status Monitoring

```systemverilog
process test_processes[3];

task monitor_processes();
    forever begin
        #10;
        $display("[%0t] Process Status Report:", $time);
        for (int i = 0; i < 3; i++) begin
            if (test_processes[i] != null) begin
                case (test_processes[i].status())
                    process::RUNNING:   $display("  Process %0d: RUNNING", i);
                    process::WAITING:   $display("  Process %0d: WAITING", i);
                    process::SUSPENDED: $display("  Process %0d: SUSPENDED", i);
                    process::KILLED:    $display("  Process %0d: KILLED", i);
                    process::FINISHED:  $display("  Process %0d: FINISHED", i);
                endcase
            end
        end
        $display("");
    end
endtask

initial begin
    fork
        // Start test processes
        for (int i = 0; i < 3; i++) begin
            automatic int proc_id = i;
            fork
                begin
                    test_processes[proc_id] = process::self();
                    $display("[%0t] Process %0d starting", $time, proc_id);
                    #($urandom_range(20, 50));
                    $display("[%0t] Process %0d finished", $time, proc_id);
                end
            join_none
        end
        
        monitor_processes();
        
        // Control logic
        begin
            #25;
            if (test_processes[1] != null)
                test_processes[1].suspend();
            
            #15;
            if (test_processes[0] != null)
                test_processes[0].kill();
            
            #10;
            if (test_processes[1] != null)
                test_processes[1].resume();
        end
    join_any
    
    #100;
    $finish;
end
```

#### Complete Communication Example

Here's a comprehensive example demonstrating multiple communication and synchronization mechanisms:

```systemverilog
// Data structures
typedef struct packed {
    int id;
    bit [31:0] data;
    bit [7:0] priority;
} message_t;

// Communication objects
mailbox #(message_t) msg_queue;
semaphore bus_semaphore;
event config_done, test_start, test_complete;

// Producer class
class Producer;
    int producer_id;
    
    function new(int id);
        this.producer_id = id;
    endfunction
    
    task run();
        message_t msg;
        for (int i = 0; i < 5; i++) begin
            msg.id = producer_id * 100 + i;
            msg.data = $urandom();
            msg.priority = $urandom_range(1, 10);
            
            msg_queue.put(msg);
            $display("[%0t] Producer %0d: Sent message %0d (priority=%0d)", 
                     $time, producer_id, msg.id, msg.priority);
            #($urandom_range(5, 15));
        end
    endtask
endclass

// Consumer class
class Consumer;
    int consumer_id;
    
    function new(int id);
        this.consumer_id = id;
    endfunction
    
    task run();
        message_t msg;
        repeat(7) begin  // Process some messages
            msg_queue.get(msg);
            
            // Acquire bus for processing
            bus_semaphore.get(1);
            $display("[%0t] Consumer %0d: Processing message %0d (priority=%0d)", 
                     $time, consumer_id, msg.id, msg.priority);
            #($urandom_range(3, 8));  // Processing time
            bus_semaphore.put(1);
            
            $display("[%0t] Consumer %0d: Completed message %0d", 
                     $time, consumer_id, msg.id);
        end
    endtask
endclass

// Main test
initial begin
    Producer prod1, prod2;
    Consumer cons1, cons2;
    
    // Initialize communication objects
    msg_queue = new();
    bus_semaphore = new(2);  // 2 bus resources
    
    // Create objects
    prod1 = new(1);
    prod2 = new(2);
    cons1 = new(1);
    cons2 = new(2);
    
    $display("[%0t] Starting communication test", $time);
    
    fork
        // Configuration phase
        begin
            $display("[%0t] Configuration starting", $time);
            #5;
            -> config_done;
            $display("[%0t] Configuration complete", $time);
        end
        
        // Wait for configuration, then start test
        begin
            @(config_done);
            -> test_start;
            $display("[%0t] Test phase starting", $time);
        end
        
        // Producers (wait for test start)
        begin
            @(test_start);
            fork
                prod1.run();
                prod2.run();
            join
            $display("[%0t] All producers finished", $time);
        end
        
        // Consumers (wait for test start)
        begin
            @(test_start);
            fork
                cons1.run();
                cons2.run();
            join
            $display("[%0t] All consumers finished", $time);
            -> test_complete;
        end
        
        // Watchdog
        begin
            #200;
            $display("[%0t] Watchdog timeout!", $time);
            -> test_complete;
        end
    join_any
    
    @(test_complete);
    $display("[%0t] Test completed", $time);
    
    // Display final statistics
    $display("Messages remaining in queue: %0d", msg_queue.num());
    
    $finish;
end
```

#### Best Practices

##### Communication and Synchronization Guidelines

1. **Choose appropriate mechanisms**: Use mailboxes for data passing, semaphores for resource control, and events for synchronization signals
2. **Avoid race conditions**: Always use proper synchronization when accessing shared resources
3. **Handle blocking operations**: Consider using non-blocking operations where appropriate to avoid deadlocks
4. **Resource cleanup**: Ensure proper release of semaphore keys and cleanup of communication objects
5. **Timeout mechanisms**: Implement timeouts for potentially blocking operations
6. **Process lifecycle management**: Properly manage process creation, suspension, and termination

This chapter covers the essential communication and synchronization mechanisms in SystemVerilog. These constructs are fundamental for building robust, coordinated testbenches and verification environments that can handle complex parallel operations while maintaining proper synchronization and data integrity.